In [1]:
import pandas as pd
import nltk
import sklearn
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, SnowballStemmer
from utils_processor.processor import Processor
import re 
from sklearn.model_selection import train_test_split

processor_ = Processor()

# Processing Pipeline

## Realizar carga de dataset 20N

Se carga el dataset denominado "Multi-Domain Sentiment Dataset" mediante pansas y se elimina la columna "Unnamed", la cual no va a aportar al posterior análisis de sentimientos

In [2]:
df_20N = pd.read_csv("data/20N_unified.csv")

In [3]:
df_20N.drop(columns=["Unnamed: 0"],inplace=True)

In [4]:
df_20N.head()

,category,email_id,from,subject,body
0,alt.atheism,49960,mathew <mathew@mantis.co.uk>,Alt.Atheism FAQ: Atheist Resources,Archive-name: atheism/resources Alt-atheism-ar...
1,alt.atheism,51060,mathew <mathew@mantis.co.uk>,Alt.Atheism FAQ: Introduction to Atheism,Archive-name: atheism/introduction Alt-atheism...
2,alt.atheism,51119,I3150101@dbstu1.rz.tu-bs.de (Benedikt Rosenau),Re: Gospel Dating,In article <65974@mimsy.umd.edu> mangoe@cs.umd...
3,alt.atheism,51120,mathew <mathew@mantis.co.uk>,Re: university violating separation of church/...,dmn@kepler.unh.edu (...until kings become phil...
4,alt.atheism,51121,strom@Watson.Ibm.Com (Rob Strom),"Re: [soc.motss, et al.] ""Princeton axes matchi...",In article <N4HY.93Apr5120934@harder.ccr-p.ida...


In [5]:
df_20N.dtypes

category    object
email_id     int64
from        object
subject     object
body        object
dtype: object

Se realiza una generalización de todos los e-mails por el caracter E-MAIL para evitar errores como detectar este tipo de caracteres como sentimiento

In [6]:
def replace_email(dataFrame, category):
    """
    Replaces all emails that contain an "@" and replaces them with "EMAIL".
    """
    # Evitar el warning SettingWithCopy haciendo una copia del DataFrame
    dataFrame = dataFrame.copy()
    
    # Trabajamos sobre la columna específica
    column_data = dataFrame[category].copy()

    for i in range(len(column_data)):
        text = column_data[i]
        
        # Dividimos el texto en palabras
        words = text.split()
        
        for j in words:
            if "@" in j:
                # Encontramos la palabra completa y la reemplazamos por "EMAIL"
                text = text.replace(j, "EMAIL")
        
        # Guardamos el texto actualizado
        column_data[i] = text

    return column_data

Se utiliza esta función para añadir espacios cuando se realiza la eliminación de caracteres especiales en la normalización de textos

In [7]:
def add_spaces_around_special_chars(text):
    """
    Adds spaces around '-' and '/' in the text.
    """
    # Usamos re.sub para añadir espacios alrededor de '-' y '/'
    text = re.sub(r'([-/_.:])', r' \1 ', text)
    
    # Reemplazamos múltiples espacios por un solo espacio
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

Aplicar todas las funciones en el dataset

In [8]:
df_20N["from"] = replace_email(df_20N,"from").apply(add_spaces_around_special_chars)
df_20N["body"] = replace_email(df_20N,"body").apply(add_spaces_around_special_chars)
df_20N["subject"] = replace_email(df_20N,"subject").apply(add_spaces_around_special_chars)



In [9]:
df_20N

,category,email_id,from,subject,body
0,alt.atheism,49960,mathew EMAIL,Alt . Atheism FAQ : Atheist Resources,Archive - name : atheism / resources Alt - ath...
1,alt.atheism,51060,mathew EMAIL,Alt . Atheism FAQ : Introduction to Atheism,Archive - name : atheism / introduction Alt - ...
2,alt.atheism,51119,EMAIL (Benedikt Rosenau),Re : Gospel Dating,In article EMAIL EMAIL (Charley Wingate) write...
3,alt.atheism,51120,mathew EMAIL,Re : university violating separation of church...,EMAIL ( . . . until kings become philosophers ...
4,alt.atheism,51121,EMAIL (Rob Strom),"Re : [soc . motss, et al . ] ""Princeton axes m...",In article EMAIL EMAIL (Bob McGwier) writes : ...
...,...,...,...,...,...
18789,talk.religion.misc,84564,EMAIL (Stephen Buckley),Re : Religion and marriage,EMAIL (Paul D Boxrud) writes : > I wasn't sure...
18790,talk.religion.misc,84565,EMAIL (Jon Baker),Re : A Message for you Mr . President : How do...,In article EMAIL EMAIL (Joakim Ruud) writes : ...
18791,talk.religion.misc,84568,EMAIL (Paul Harvey),Re : Why did they behave as they did (Waco - -...,In article EMAIL EMAIL (Bill Jefferys) writes ...
18792,talk.religion.misc,84569,EMAIL,Re : Info about New Age!,In article EMAIL EMAIL (Gerry Palo) >The dange...


Extraer características de los e-mails

In [10]:
df_20N["from"] = df_20N["from"].apply(lambda x: processor_.preprocessing_pipeline(x))
df_20N["body"] = df_20N["body"].apply(lambda x: processor_.preprocessing_pipeline(x))
df_20N["subject"] = df_20N["subject"].apply(lambda x: processor_.preprocessing_pipeline(x))


In [11]:
df_20N

,category,email_id,from,subject,body
0,alt.atheism,49960,mathew email,alt atheism faq atheist resourc,archiv name atheism resourc alt atheism archiv...
1,alt.atheism,51060,mathew email,alt atheism faq introduct atheism,archiv name atheism introduct alt atheism arch...
2,alt.atheism,51119,email benedikt rosenau,gospel date,articl email email charley wingat write well j...
3,alt.atheism,51120,mathew email,univers violat separ church state,email king becom philosoph philosoph becom kin...
4,alt.atheism,51121,email rob strom,soc motss et al princeton axe match fund boy s...,articl email email bob mcgwier write num howev...
...,...,...,...,...,...
18789,talk.religion.misc,84564,email stephen buckley,religion marriag,email paul boxrud write nt sure right newsgrou...
18790,talk.religion.misc,84565,email jon baker,messag mr presid know happen,articl email email joakim ruud write articl em...
18791,talk.religion.misc,84568,email paul harvey,behav waco read suggest,articl email email bill jefferi write would li...
18792,talk.religion.misc,84569,email,info new age,articl email email gerri palo danger anti cult...


## Dividir datos de entrenamiento

Se aplican diferentes métodos de vectorización de textos para proceder con el análisis de sentimientos

In [12]:
df_20N['text'] = df_20N.apply(lambda row: f"{row['from']} {row['subject']} {row['body']}", axis=1)
df_20N.drop(columns=['from','subject','body','email_id'],inplace=True)
# Verifica los resultados


In [13]:
df_20N.columns

Index(['category', 'text'], dtype='object')

In [14]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, ConfusionMatrixDisplay
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV


### CountVectorizer

Se aplica el método de vectorización conocido como CountVectorizer, el cual funciona de forma similar a una bolsa de palabras (BOW)

In [15]:
texto_procesado = df_20N["text"].tolist()

In [16]:
vectorizer = CountVectorizer(max_features=5000, stop_words=stopwords.words('english'))
texto_features_vectorizer = vectorizer.fit_transform(texto_procesado).toarray()

In [17]:
vectorizer.get_feature_names_out()

array(['0d', '0el', '0g', ..., 'zoolog', 'zoom', 'zx'], dtype=object)

In [18]:
len(vectorizer.get_feature_names_out().tolist())

5000

In [19]:
texto_procesado[2895].split()[5]

'articl'

### Tf-idf

Se aplica el método de vectorización conocido como Tf-Idf, la cual asigna una ponderación a cada palabra en el documento que se encuentra

In [20]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words=stopwords.words('english'))
texto_features_tfidf_vectorizer = tfidf_vectorizer.fit_transform(texto_procesado).toarray()


In [21]:
len(tfidf_vectorizer.get_feature_names_out().tolist())

5000

In [22]:
labels = df_20N['category'].values
labels

array(['alt.atheism', 'alt.atheism', 'alt.atheism', ...,
       'talk.religion.misc', 'talk.religion.misc', 'talk.religion.misc'],
      dtype=object)

Se procede a realizar cada división de variables en 60% entrenamiento, 10% validación y 30% prueba

In [23]:

# Primera división: 70% (entrenamiento + validación) y 30% (prueba)
X_temp_v, X_test_v, y_temp_v, y_test_v = train_test_split(texto_features_vectorizer, labels, test_size=0.3, random_state=0)

# Segunda división: del 70%, tomamos 60% para entrenamiento y 10% para validación
# 0.14285 es aproximadamente 10% / 70%, que nos da el 10% del conjunto de datos original
X_train_v, X_val_v, y_train_v, y_val_v = train_test_split(X_temp_v, y_temp_v, test_size=0.1/0.7, random_state=0)

# Repetimos el proceso para los datos TF-IDF
X_temp_tf, X_test_tf, y_temp_tf, y_test_tf = train_test_split(texto_features_tfidf_vectorizer, labels, test_size=0.3, random_state=0)
X_train_tf, X_val_tf, y_train_tf, y_val_tf = train_test_split(X_temp_tf, y_temp_tf, test_size=0.1/0.7, random_state=0)

In [24]:
models = []

## Clasificador Multinomial de Naive Bayes

### CountVectorizer

Se realiza el análisis de sentimientos mediante el clasificador multinomial de Naive Bayes

Se realiza el entrenamiento de los modelos sobre los datos de entrenamiento

In [25]:
nb = MultinomialNB()
# El método fit en sklearn permite ejecutar el proceso de entrenamiento.
nb.fit(X_train_v, y_train_v)

MultinomialNB()

Se realiza la predicción sobre los datos de prueba

In [26]:
predictions = nb.predict(X_test_v)

In [27]:
print(accuracy_score(y_test_v, predictions))

0.8001418691257315


Se aplica la predicción sobre los datos de validación

In [28]:
predictions_validation = nb.predict(X_val_v)

In [29]:
print(accuracy_score(y_val_v, predictions_validation))

0.8127659574468085


### Matriz Tf-idf

Se realiza el análisis de sentimientos mediante el clasificador multinomial de Naive Bayes.


Entrenamiento sobre la división de datos de entrenamiento

In [30]:
nb_tf = MultinomialNB()
# El método fit en sklearn permite ejecutar el proceso de entrenamiento.
nb_tf.fit(X_train_tf, y_train_tf)

MultinomialNB()

Predicciones en los datos de prueba

In [31]:
predictions_tf = nb_tf.predict(X_test_tf)

In [32]:
print(accuracy_score(y_test_tf, predictions))

0.8001418691257315


Predicciones en los datos de validación

In [33]:
predictions_tf_validation = nb_tf.predict(X_val_tf)

In [34]:
print(accuracy_score(y_val_tf, predictions_tf_validation))

0.85


## Regresion Logistica

In [35]:
from sklearn.linear_model import LogisticRegression

### CountVectorizer

Se realiza la clasificación de sentimientos mediante un modelo de regresión logística

Entrenamiento sobre la división de datos de entrenamiento

In [36]:
reg = LogisticRegression()

reg.fit(X_train_v,y_train_v)

C:\Users\USUARIO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

Predicción sobre los datos de prueba

In [37]:
predictions_reg_v = reg.predict(X_test_v)
predictions_reg_v

array(['sci.electronics', 'comp.windows.x', 'rec.sport.baseball', ...,
       'sci.electronics', 'misc.forsale', 'sci.crypt'], dtype=object)

In [38]:
print(accuracy_score(y_test_v, predictions_reg_v))

0.8152154637347048


Predicción sobre los datos de validación

In [39]:
predictions_validation_reg_v = reg.predict(X_val_v)

In [40]:
print(accuracy_score(y_val_v, predictions_validation_reg_v))

0.801063829787234


### Matriz Tf-idf

Se realiza la clasificación de sentimientos mediante un modelo de regresión logística

Entrenamiento sobre la división de datos de entrenamiento

In [41]:
reg_tf = LogisticRegression()

reg_tf.fit(X_train_tf,y_train_tf)

LogisticRegression()

Predicción sobre conjuntos de prueba

In [42]:
predictions_reg_tf = reg_tf.predict(X_test_tf)
predictions_reg_tf

array(['sci.electronics', 'comp.windows.x', 'rec.sport.baseball', ...,
       'sci.electronics', 'comp.sys.ibm.pc.hardware', 'sci.crypt'],
      dtype=object)

In [43]:
print(accuracy_score(y_test_tf, predictions_reg_tf))

0.8632736300762547


Predicción sobre conjuntos de validación

In [44]:
predictions_validation_reg_tf = reg_tf.predict(X_val_tf)
predictions_validation_reg_tf

array(['sci.med', 'talk.politics.mideast', 'rec.sport.hockey', ...,
       'rec.autos', 'sci.space', 'talk.politics.guns'], dtype=object)

In [45]:
print(accuracy_score(y_val_tf, predictions_validation_reg_tf))

0.8627659574468085


tiene sentido porque el tfidf le aporta mas valor a los terminos mas diferentes

## Cross Validation 

Cross validation consiste en separar y entrenar el dataset en k diferentes folds para poder evitar el overfitting  seleccionando un k como datos de validacion y los otros k como los datos de entrenamiento, mide la capacidad del modelo para generalizar datos no vistos en el entrenamiento.

In [46]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [47]:
import pandas as pd

# Convertir los arrays a DataFrames antes de concatenarlos
X_train_val_v = pd.DataFrame(X_train_v)
X_val_v_df = pd.DataFrame(X_val_v)
X_train_val_v = pd.concat([X_train_val_v, X_val_v_df])

y_train_val_v = pd.Series(y_train_v)
y_val_v_series = pd.Series(y_val_v)
y_train_val_v = pd.concat([y_train_val_v, y_val_v_series])

X_train_val_tf = pd.DataFrame(X_train_tf)
X_val_tf_df = pd.DataFrame(X_val_tf)
X_train_val_tf = pd.concat([X_train_val_tf, X_val_tf_df])

y_train_val_tf = pd.Series(y_train_tf)
y_val_tf_series = pd.Series(y_val_tf)
y_train_val_tf = pd.concat([y_train_val_tf, y_val_tf_series])

# Definir los modelos con los nuevos conjuntos combinados
models = {
    "Vectorizer Naive Bayes": (nb, X_train_val_v),
    "Tf-Idf Naive Bayes": (nb_tf, X_train_val_tf),
    "Vectorizer LR": (reg, X_train_val_v),
    "Tf-Idf LR": (reg_tf, X_train_val_tf)
}

models

{'Vectorizer Naive Bayes': (MultinomialNB(),
        0     1     2     3     4     5     6     7     8     9     ...  4990  \
  0        0     0     0     0     0     0     0     0     0     0  ...     0   
  1        0     0     0     0     0     0     0     0     0     0  ...     0   
  2        0     0     0     0     0     0     0     0     0     0  ...     0   
  3        0     0     0     0     0     0     0     0     0     0  ...     0   
  4        0     0     0     0     0     0     0     0     0     0  ...     0   
  ...    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
  1875     0     0     0     0     0     0     0     0     0     0  ...     0   
  1876     0     0     0     0     0     0     0     0     0     0  ...     0   
  1877     0     0     0     0     0     0     0     0     0     0  ...     0   
  1878     0     0     0     0     0     0     0     0     0     0  ...     0   
  1879     0     0     0     0     0     0     0     0     0    

### Naive Bayes Vectorizer

Se realiza una validación cruzada con el vectorizador de Naive Bayes y se compara mediante las métricas de precisión, accuracy, f1_score (para todos los casos macro y micro al ser una clasificación multiclase), media y desviación estándar

In [48]:
def cross_val_report(model, X, y, model_name, cv=10):
    """
    Performs cross-validation on the provided model and calculates various evaluation metrics, 
    including accuracy, precision, recall, and F1 score (both macro and micro averages).

    Args:
        model: The machine learning model to be evaluated.
        X (ndarray or DataFrame): The input features for the model.
        y (ndarray or Series): The true labels for the input data.
        model_name (str): The name of the model being evaluated (for display purposes).
        cv (int): The number of cross-validation folds (default is 10).

    Returns:
        None: The function prints the mean and standard deviation of the evaluation metrics across the folds.
    """
    # Definir las métricas que queremos evaluar
    scoring = {
        'accuracy': 'accuracy',
        'precision_macro': make_scorer(precision_score, average='macro'),
        'recall_macro': make_scorer(recall_score, average='macro'),
        'f1_macro': make_scorer(f1_score, average='macro'),
        'precision_micro': make_scorer(precision_score, average='micro'),
        'recall_micro': make_scorer(recall_score, average='micro'),
        'f1_micro': make_scorer(f1_score, average='micro')
    }
    
    # Realizar validación cruzada
    scores = cross_validate(model, X, y, cv=cv, scoring=scoring)
    
    # Crear un DataFrame con los resultados
    results = pd.DataFrame({
        'Metric': ['Accuracy', 'Precision (Macro)', 'Recall (Macro)', 'F1 (Macro)',
                   'Precision (Micro)', 'Recall (Micro)', 'F1 (Micro)'],
        'Mean': [scores[f'test_{metric}'].mean() for metric in scoring.keys()],
        'Std Dev': [scores[f'test_{metric}'].std() for metric in scoring.keys()]
    })
    
    # Imprimir resultados
    print(f"\nResultados de Validación Cruzada para {model_name} (CV={cv}):")
    print(results.to_string(index=False))

In [49]:
for model_name, (model, X) in models.items():
    if model_name.startswith("Vectorizer"):
        cross_val_report(model, X, y_train_val_v, model_name)


Resultados de Validación Cruzada para Vectorizer Naive Bayes (CV=10):
           Metric     Mean  Std Dev
         Accuracy 0.802354 0.012832
Precision (Macro) 0.821360 0.011898
   Recall (Macro) 0.798145 0.012999
       F1 (Macro) 0.784460 0.013161
Precision (Micro) 0.802354 0.012832
   Recall (Micro) 0.802354 0.012832
       F1 (Micro) 0.802354 0.012832


C:\Users\USUARIO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\USUARIO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/


Resultados de Validación Cruzada para Vectorizer LR (CV=10):
           Metric     Mean  Std Dev
         Accuracy 0.820445 0.017444
Precision (Macro) 0.821359 0.016898
   Recall (Macro) 0.815880 0.017040
       F1 (Macro) 0.816378 0.017314
Precision (Micro) 0.820445 0.017444
   Recall (Micro) 0.820445 0.017444
       F1 (Micro) 0.820445 0.017444


C:\Users\USUARIO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [50]:
for model_name, (model, X) in models.items():
    if not model_name.startswith("Vectorizer"):
        cross_val_report(model, X, y_train_val_v, model_name)



Resultados de Validación Cruzada para Tf-Idf Naive Bayes (CV=10):
           Metric     Mean  Std Dev
         Accuracy 0.844164 0.009148
Precision (Macro) 0.858696 0.010853
   Recall (Macro) 0.832549 0.009297
       F1 (Macro) 0.830747 0.009832
Precision (Micro) 0.844164 0.009148
   Recall (Micro) 0.844164 0.009148
       F1 (Micro) 0.844164 0.009148

Resultados de Validación Cruzada para Tf-Idf LR (CV=10):
           Metric     Mean  Std Dev
         Accuracy 0.862105 0.009381
Precision (Macro) 0.863426 0.010442
   Recall (Macro) 0.856793 0.009855
       F1 (Macro) 0.857923 0.010011
Precision (Micro) 0.862105 0.009381
   Recall (Micro) 0.862105 0.009381
       F1 (Micro) 0.862105 0.009381


### LR Vectorizer

Se realiza una validación cruzada con el LR utilizando CountVectorizer y se compara mediante las métricas de precisión, accuracy, f1_score (para todos los casos macro y micro al ser una clasificación multiclase), media y desviación estándar

In [51]:
from sklearn.metrics import make_scorer, f1_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, cross_validate
import pandas as pd

# Definir el espacio de búsqueda de hiperparámetros
param_grid = {'max_iter': [100, 500, 1000]}

# Crear el objeto Logistic Regression (sin el parámetro 'multi_class')
reg = LogisticRegression(max_iter=1000, solver='lbfgs')

# Crear el objeto GridSearchCV con f1_macro
f1_scorer = make_scorer(f1_score, average='macro')
grid_search = GridSearchCV(estimator=reg, param_grid=param_grid, 
                           scoring=f1_scorer, cv=10, n_jobs=1, verbose=2)

# Asegúrate de que los datos combinados están en el formato adecuado
# X_train_val_v y y_train_val_v ya están combinados

# Realizar la búsqueda de hiperparámetros
grid_search.fit(X_train_val_v, y_train_val_v)

# Imprimir los mejores parámetros y el mejor puntaje
print("Mejores parámetros encontrados:")
print(grid_search.best_params_)
print("\nMejor puntaje F1 (macro):")
print(grid_search.best_score_)

best_lr = grid_search.best_estimator_

cross_val_report(best_lr, X_train_val_v, y_train_val_v, "Mejor Logistic Regression")


Fitting 10 folds for each of 3 candidates, totalling 30 fits


C:\Users\USUARIO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\USUARIO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\USUARIO\AppData\Local\Packages\P

[CV] END .......................................max_iter=100; total time=  15.7s


C:\Users\USUARIO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\USUARIO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\USUARIO\AppData\Local\Packages\P

[CV] END .......................................max_iter=100; total time=  16.3s


...

C:\Users\USUARIO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Mejores parámetros encontrados:
{'max_iter': 100}

Mejor puntaje F1 (macro):
nan


C:\Users\USUARIO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\USUARIO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/


Resultados de Validación Cruzada para Mejor Logistic Regression (CV=10):
           Metric     Mean  Std Dev
         Accuracy 0.820445 0.017444
Precision (Macro) 0.821359 0.016898
   Recall (Macro) 0.815880 0.017040
       F1 (Macro) 0.816378 0.017314
Precision (Micro) 0.820445 0.017444
   Recall (Micro) 0.820445 0.017444
       F1 (Micro) 0.820445 0.017444


C:\Users\USUARIO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### LR Tf

Se realiza una validación cruzada utilizando LR con la matriz Tf y se compara mediante las métricas de precisión, accuracy, f1_score (para todos los casos macro y micro al ser una clasificación multiclase), media y desviación estándar

In [52]:
from sklearn.metrics import make_scorer, f1_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, cross_validate
import pandas as pd

# Definir el espacio de búsqueda de hiperparámetros
param_grid = {'max_iter': [100, 500, 1000]}

# Crear el objeto Logistic Regression (sin el parámetro 'multi_class')
reg_tf= LogisticRegression(max_iter=1000, solver='lbfgs')

# Crear el objeto GridSearchCV con f1_macro
f1_scorer = make_scorer(f1_score, average='macro')
grid_search_tf = GridSearchCV(estimator=reg_tf, param_grid=param_grid, 
                           scoring=f1_scorer, cv=10, n_jobs=1, verbose=2)

# Asegúrate de que los datos combinados están en el formato adecuado
# X_train_val_v y y_train_val_v ya están combinados

# Realizar la búsqueda de hiperparámetros
grid_search_tf.fit(X_train_val_tf, y_train_val_tf)

# Imprimir los mejores parámetros y el mejor puntaje
print("Mejores parámetros encontrados:")
print(grid_search_tf.best_params_)
print("\nMejor puntaje F1 (macro):")
print(grid_search_tf.best_score_)

best_lr_tf = grid_search_tf.best_estimator_

cross_val_report(best_lr_tf, X_train_val_tf, y_train_val_tf, "Mejor Logistic Regression TF-IDF")


Fitting 10 folds for each of 3 candidates, totalling 30 fits


C:\Users\USUARIO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\USUARIO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\USUARIO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

[CV] END .......................................max_iter=100; total time=   6.5s


C:\Users\USUARIO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\USUARIO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\USUARIO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

[CV] END .......................................max_iter=100; total time=   5.0s


...

[CV] END ......................................max_iter=1000; total time=   6.7s


C:\Users\USUARIO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\USUARIO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\USUARIO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

[CV] END ......................................max_iter=1000; total time=   6.7s


C:\Users\USUARIO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\USUARIO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\USUARIO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

[CV] END ......................................max_iter=1000; total time=   6.6s
Mejores parámetros encontrados:
{'max_iter': 100}

Mejor puntaje F1 (macro):
nan

Resultados de Validación Cruzada para Mejor Logistic Regression TF-IDF (CV=10):
           Metric     Mean  Std Dev
         Accuracy 0.862105 0.009381
Precision (Macro) 0.863426 0.010442
   Recall (Macro) 0.856793 0.009855
       F1 (Macro) 0.857923 0.010011
Precision (Micro) 0.862105 0.009381
   Recall (Micro) 0.862105 0.009381
       F1 (Micro) 0.862105 0.009381


## Evaluation

Se hace una comparativa final y se escoge el modelo con mejores métricas, haciendo una prorización de la métrica f1_score

In [53]:
# Evaluar el mejor modelo en el conjunto de test
def evaluate_model(model, X_test, y_test, model_name):
    """
    Evaluates the provided model on the test set and computes precision, recall, and F1-score 
    using both macro and micro averages.

    Args:
        model: The trained machine learning model to be evaluated.
        X_test (ndarray or DataFrame): The input features for the test set.
        y_test (ndarray or Series): The true labels for the test set.
        model_name (str): The name of the model being evaluated (for display purposes).

    Returns:
        dict: A dictionary containing the model name and the computed precision, recall, and F1-scores 
              for both macro and micro averages.
    """
    
    # Predecir las etiquetas para el conjunto de test
    y_pred = model.predict(X_test)
    
    # Calcular precision, recall y F1 con promedio macro y micro
    precision_macro = precision_score(y_test, y_pred, average='macro')
    recall_macro = recall_score(y_test, y_pred, average='macro')
    f1_macro = f1_score(y_test, y_pred, average='macro')
    
    precision_micro = precision_score(y_test, y_pred, average='micro')
    recall_micro = recall_score(y_test, y_pred, average='micro')
    f1_micro = f1_score(y_test, y_pred, average='micro')
    
    # Imprimir los resultados
    print(f"\nEvaluación del Modelo: {model_name}")
    print(f"Precision (Macro): {precision_macro:.4f}")
    print(f"Recall (Macro): {recall_macro:.4f}")
    print(f"F1 Score (Macro): {f1_macro:.4f}")
    print(f"Precision (Micro): {precision_micro:.4f}")
    print(f"Recall (Micro): {recall_micro:.4f}")
    print(f"F1 Score (Micro): {f1_micro:.4f}")
    
    return {
        'model_name': model_name,
        'precision_macro': precision_macro,
        'recall_macro': recall_macro,
        'f1_macro': f1_macro,
        'precision_micro': precision_micro,
        'recall_micro': recall_micro,
        'f1_micro': f1_micro
    }

# Evaluar los modelos con el conjunto de test
test_results = []

# Evaluar el mejor modelo Logistic Regression con vectorizer
test_results.append(evaluate_model(best_lr, X_test_v, y_test_v, "Best Logistic Regression (Vectorizer)"))

# Evaluar el mejor modelo Logistic Regression con tf-idf
test_results.append(evaluate_model(best_lr_tf, X_test_tf, y_test_tf, "Best Logistic Regression (Tf-Idf)"))

# Evaluar Naive Bayes con vectorizer
test_results.append(evaluate_model(nb, X_test_v, y_test_v, "Naive Bayes (Vectorizer)"))

# Evaluar Naive Bayes con tf-idf
test_results.append(evaluate_model(nb_tf, X_test_tf, y_test_tf, "Naive Bayes (Tf-Idf)"))

# Comparar los modelos y encontrar el mejor
best_model = max(test_results, key=lambda x: x['f1_macro'])

print(f"\nEl mejor modelo es: {best_model['model_name']}")
print(f"Con un F1 Score (Macro) de: {best_model['f1_macro']:.4f}")


Evaluación del Modelo: Best Logistic Regression (Vectorizer)
Precision (Macro): 0.8299
Recall (Macro): 0.8295
F1 Score (Macro): 0.8285
Precision (Micro): 0.8338
Recall (Micro): 0.8338
F1 Score (Micro): 0.8338

Evaluación del Modelo: Best Logistic Regression (Tf-Idf)
Precision (Macro): 0.8692
Recall (Macro): 0.8640
F1 Score (Macro): 0.8654
Precision (Micro): 0.8718
Recall (Micro): 0.8718
F1 Score (Micro): 0.8718

Evaluación del Modelo: Naive Bayes (Vectorizer)
Precision (Macro): 0.7978
Recall (Macro): 0.7946
F1 Score (Macro): 0.7780
Precision (Micro): 0.8001
Recall (Micro): 0.8001
F1 Score (Micro): 0.8001

Evaluación del Modelo: Naive Bayes (Tf-Idf)
Precision (Macro): 0.8567
Recall (Macro): 0.8301
F1 Score (Macro): 0.8295
Precision (Micro): 0.8473
Recall (Micro): 0.8473
F1 Score (Micro): 0.8473

El mejor modelo es: Best Logistic Regression (Tf-Idf)
Con un F1 Score (Macro) de: 0.8654
